In [1]:
import numpy as np
import theano
import theano.tensor as T
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from theano.tensor.shared_randomstreams import RandomStreams
from util import relu, error_rate, getKaggleMNIST, init_weights

In [19]:
class RBM(object):
    def __init__(self, M, an_id):
        self.M = M
        self.id = an_id
        self.rng = RandomStreams()
    
    def fit(self, X, learning_rate=0.1, epochs=10, batch_sz=100, show_fig=False):
        N, D = X.shape
        n_batches = int(N/batch_sz)
        
        W0 = init_weights((D, self.M))
        self.W = theano.shared(W0, 'W_%s' %self.id)
        self.c = theano.shared(np.zeros(self.M), 'c_%s' %self.id)
        self.b = theano.shared(np.zeros(D), 'b_%s' %self.id)
        self.params = [self.W, self.c, self.b]
        self.forward_params = [self.W, self.c]
        
        self.dW = theano.shared(np.zeros(W0.shape), 'dW_%s' %self.id )
        self.dc = theano.shared(np.zeros(self.M), 'dbh_%s' %self.id)
        self.db = theano.shared(np.zeros(D), 'db0_%s' %self.id)
        self.dparams = [self.dW, self.dc, self.db]
        self.forward_dparams = [self.dW, self.dc]
        
        X_in = T.matrix('X_%s' %self.id)
        H = T.nnet.sigmoid(X_in.dot(self.W)+self.c)
        self.hidden_op = theano.function(
            inputs=[X_in],
            outputs=H
        )
        
        X_hat = self.forward_output(X_in)
        cost = -(X_in*T.log(X_hat)+(1-X_in)*T.log(1-X_hat)).sum()/N
        cost_op = theano.function(
            inputs=[X_in],
            outputs=cost
        )
        
        H_sample = self.sample_h_given_v(X_in)
        X_sample = self.sample_v_given_h(H_sample)
        
        objective = T.mean(self.free_energy(X_in)) - T.mean(self.free_energy(X_sample))
        
        updates = [(p, p - learning_rate*T.grad(objective, p, consider_constant=[X_sample])) for p in self.params]
        train_op = theano.function(
            inputs = [X_in],
            updates=updates
        )
        
        costs=[]
        print('training rbm %s' %self.id)
        for i in range(epochs):
            print('epoch:', i)
            X = shuffle(X)
            for j in range(n_batches):
                batch = X[j*batch_sz:(j+1)*batch_sz]
                train_op(batch)
                the_cost = cost_op(X)
                print("batch", j, "/", n_batches, "cost:", the_cost)
                costs.append(the_cost)
            if show_fig:
                plt.plot(costs)
                plt.show()
            
    def free_energy(self, V):
        return -V.dot(self.b) - T.sum(T.log(1+T.exp(V.dot(self.W)+self.c)), axis=1)
    
    def sample_h_given_v(self, V):
        p_h_given_v = T.nnet.sigmoid(V.dot(self.W)+self.c)
        h_sample = self.rng.binomial(size=p_h_given_v.shape, n=1, p=p_h_given_v)
        return h_sample
    
    def sample_v_given_h(self, H):
        p_v_given_h = T.nnet.sigmoid(H.dot(self.W.T) +self.b)
        v_sample = self.rng.binomial(size=p_v_given_h.shape, n=1, p=p_v_given_h)
        return v_sample
    
    def forward_hidden(self, X):
        return T.nnet.sigmoid(X.dot(self.W)+self.c)
    
    def forward_output(self, X):
        Z = self.forward_hidden(X)
        Y = T.nnet.sigmoid(Z.dot(self.W.T)+self.b)
        return Y
    

In [20]:
class DNN(object):
    def __init__(self, hidden_layer_sizes, UnsupervisedModel=RBM):
        self.hidden_layers = []
        count = 0
        for M in hidden_layer_sizes:
            ae = UnsupervisedModel(M, count)
            self.hidden_layers.append(ae)
            count += 1


    def fit(self, X, Y, Xtest, Ytest, pretrain=True, learning_rate=0.01, mu=0.99, reg=0.1, epochs=1, batch_sz=100):
        # greedy layer-wise training of autoencoders
        pretrain_epochs = 1
        if not pretrain:
            pretrain_epochs = 0

        current_input = X
        for ae in self.hidden_layers:
            ae.fit(current_input, epochs=pretrain_epochs)

            # create current_input for the next layer
            current_input = ae.hidden_op(current_input)

        # initialize logistic regression layer
        N = len(Y)
        K = len(set(Y))
        W0 = init_weights((self.hidden_layers[-1].M, K))
        self.W = theano.shared(W0, "W_logreg")
        self.b = theano.shared(np.zeros(K), "b_logreg")

        self.params = [self.W, self.b]
        for ae in self.hidden_layers:
            self.params += ae.forward_params

        # for momentum
        self.dW = theano.shared(np.zeros(W0.shape), "dW_logreg")
        self.db = theano.shared(np.zeros(K), "db_logreg")
        self.dparams = [self.dW, self.db]
        for ae in self.hidden_layers:
            self.dparams += ae.forward_dparams

        X_in = T.matrix('X_in')
        targets = T.ivector('Targets')
        pY = self.forward(X_in)

        # squared_magnitude = [(p*p).sum() for p in self.params]
        # reg_cost = T.sum(squared_magnitude)
        cost = -T.mean( T.log(pY[T.arange(pY.shape[0]), targets]) ) #+ reg*reg_cost
        prediction = self.predict(X_in)
        cost_predict_op = theano.function(
            inputs=[X_in, targets],
            outputs=[cost, prediction],
        )

        updates = [
            (p, p + mu*dp - learning_rate*T.grad(cost, p)) for p, dp in zip(self.params, self.dparams)
        ] + [
            (dp, mu*dp - learning_rate*T.grad(cost, p)) for p, dp in zip(self.params, self.dparams)
        ]
        # updates = [(p, p - learning_rate*T.grad(cost, p)) for p in self.params]
        train_op = theano.function(
            inputs=[X_in, targets],
            updates=updates,
        )

        n_batches = N / batch_sz
        costs = []
        print("supervised training...")
        for i in range(epochs):
            print("epoch:", i)
            X, Y = shuffle(X, Y)
            for j in range(n_batches):
                Xbatch = X[j*batch_sz:(j*batch_sz + batch_sz)]
                Ybatch = Y[j*batch_sz:(j*batch_sz + batch_sz)]
                train_op(Xbatch, Ybatch)
                the_cost, the_prediction = cost_predict_op(Xtest, Ytest)
                error = error_rate(the_prediction, Ytest)
                print("j / n_batches:", j, "/", n_batches, "cost:", the_cost, "error:", error)
                costs.append(the_cost)
        plt.plot(costs)
        plt.show()

    def predict(self, X):
        return T.argmax(self.forward(X), axis=1)

    def forward(self, X):
        current_input = X
        for ae in self.hidden_layers:
            Z = ae.forward_hidden(current_input)
            current_input = Z

        # logistic layer
        Y = T.nnet.softmax(T.dot(current_input, self.W) + self.b)
        return Y


In [21]:
def main():
    Xtrain, Ytrain, Xtest, Ytest = getKaggleMNIST()
    dnn = DNN([100,50])
    dnn.fit(Xtrain, Ytrain, Xtest, Ytest, epochs=2)

In [22]:
main()

training rbm 0
epoch: 0
batch 0 / 410 cost: 377.8018862514167
batch 1 / 410 cost: 322.2913619891312
batch 2 / 410 cost: 316.86395562244877
batch 3 / 410 cost: 300.2166678422898
batch 4 / 410 cost: 272.9640284695749
batch 5 / 410 cost: 256.07768030433556
batch 6 / 410 cost: 249.48524050698802
batch 7 / 410 cost: 244.26591323649674
batch 8 / 410 cost: 239.30241857811473
batch 9 / 410 cost: 240.00179175094897
batch 10 / 410 cost: 243.472201517157
batch 11 / 410 cost: 239.6488202332834
batch 12 / 410 cost: 256.0539567740851
batch 13 / 410 cost: 227.73745025307255
batch 14 / 410 cost: 224.7605999123447
batch 15 / 410 cost: 226.4421805000808
batch 16 / 410 cost: 234.15467290103786
batch 17 / 410 cost: 222.13297107357366
batch 18 / 410 cost: 220.8307018363832
batch 19 / 410 cost: 223.6043515571457
batch 20 / 410 cost: 235.39433556891157
batch 21 / 410 cost: 211.0637591102951
batch 22 / 410 cost: 209.06124432800624
batch 23 / 410 cost: 206.72607562014662
batch 24 / 410 cost: 206.32929694970875

KeyboardInterrupt: 